# Notebook for training a Nueral Network to crack LPN

### Step 0: Import Packages + Choose Device

In [61]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.nn import functional as F

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


### Step 1: Create LPN Oracle

In [62]:
class LPNOracle:
    def __init__(self, secret, error_rate):
        self.secret = secret
        self.dimension = len(secret)
        self.error_rate = error_rate

    def sample(self, n_amount):
        # Create random matrix.
        A = np.random.randint(0, 2, size=(n_amount, self.dimension))
        # Add Bernoulli errors.
        e = np.random.binomial(1, self.error_rate, n_amount)
        # Compute the labels.
        b = np.mod(A @ self.secret + e, 2)
        return A, b, e
    
    def percolate(self, A, b, e):
        
        goods = np.where(e == 0)[0]
        bads = np.where(e == 1)[0]
        A_good = A[goods, :]
        A_bad = A[bads, :]
        b_good = b[goods]
        b_bad = b[bads]
        
        return A_good, b_good, A_bad, b_bad
        

### Step 2: Create Dataset Class(To load the dataset)

In [63]:
class dataset(Dataset):
  def __init__(self,x,y):
    self.x = torch.from_numpy(x).to(torch.float32)
    self.y = torch.from_numpy(y).to(torch.float32)
    self.length = self.x.shape[0]
 
  def __getitem__(self,idx):
    return self.x[idx],self.y[idx]
  def __len__(self):
    return self.length

### Step 3: Get the Training Data

In [64]:
p = 0.125
dim = 16 #CHANGE THIS
s = np.random.randint(0, 2, dim)
lpn = LPNOracle(s, p)

sample_size = 10000
A, b, e = lpn.sample(sample_size)

A_good, b_good, A_bad, b_bad = lpn.percolate(A,b,e)


trainset = dataset(A, b)
bad_trainset = dataset(A_bad, b_bad)
good_trainset = dataset(A_good, b_good)
print(trainset.__len__())
trainloader = DataLoader(trainset,batch_size=64,shuffle=False)
good_trainloader = DataLoader(good_trainset, batch_size=64,shuffle=False)
bad_trainloader = DataLoader(bad_trainset, batch_size=64,shuffle=False)

10000


In [65]:
print("A : {a}".format(a = A))
print("A shape: {a}".format(a = A.shape))
print("b : {b}".format(b = b))
print("B shape: {b}".format(b = b.shape))

A : [[1 1 1 ... 0 0 0]
 [0 1 0 ... 1 1 0]
 [0 1 1 ... 1 0 0]
 ...
 [1 0 1 ... 1 0 1]
 [0 0 0 ... 0 0 1]
 [0 1 1 ... 0 1 0]]
A shape: (10000, 16)
b : [0 0 0 ... 1 0 1]
B shape: (10000,)


### Step 4: Create the Experiment Folder + Save the Key

In [66]:
import os
import json

counter = 0

while os.path.exists("./experiment-{a}".format(a = counter)):
    
    counter = counter + 1
    
print(counter)
    
os.mkdir("./experiment-{a}".format(a = counter))

slist = s.tolist()
A_list = A.tolist()
b_list = b.tolist()
s_dict = {"s" : slist, "A" : A_list, "b" : b_list, "p" : p}

with open("./experiment-{a}/secret.json".format(a = counter), "w") as sfile:
    json.dump(s_dict, sfile)
    



34


### Step 5: Create Parametric.json

In [67]:
import json

parametric = {}

parametric["layers"] = [96];
parametric["s"] = len(s.tolist())

with open("./experiment-{a}/parametric.json".format(a = counter), "w") as file:
    
    json.dump(parametric, file)

### Step 6: Create the Nueral Network

In [68]:
class cryptnet(nn.Module):
  def __init__(self,s):
    super(cryptnet,self).__init__()
    self.fc1 = nn.Linear(s.size,32)
    self.act1 = nn.ReLU()
    self.fc2 = nn.Linear(32,32)
    self.act2 = nn.ReLU()
    self.fc3 = nn.Linear(32,1)
    self.siggy = nn.Sigmoid()
  def forward(self,x):
    x = self.act1(self.fc1(x))
    x = self.act2(self.fc2(x))
    x = self.siggy(self.fc3(x))
    return x

chicken_net = cryptnet(s)

In [69]:
import json

class parametric_cryptnet(nn.Module):
    def __init__(self, parametric):
        super(parametric_cryptnet, self).__init__()
        self.act = nn.ModuleList()
        self.fc = nn.ModuleList()
    
        layer_arr = parametric["layers"]            
        size_lay = len(layer_arr)
    
        self.fc.append(nn.Linear(parametric["s"], layer_arr[0]))
        self.act.append(nn.ReLU())
    
        for i in range(size_lay - 1):
            self.fc.append(nn.Linear(layer_arr[i], layer_arr[i + 1]))
            self.act.append(nn.ReLU())
                       
        self.fc.append(nn.Linear(layer_arr[size_lay - 1], 1))
        self.act.append(nn.Sigmoid())
        
    def forward(self,x):
        for i in range(0, len(self.fc)):
            x = self.act[i](self.fc[i](x))
        return x

                   


### Step 6.5: Training a Model on the Good Dataset

In [70]:
paracrypt_good = parametric_cryptnet(parametric)
learning_rate = 0.01
epochs = 1600
optimizer = torch.optim.SGD(paracrypt_good.parameters(),lr=learning_rate)
loss_fn = nn.BCELoss()

print("s  : {a}".format(a = s))

with open("./experiment-{a}/good_training.txt".format(a = counter), "w") as data_file:
    for i in range(epochs):
        for j,(x_train,y_train) in enumerate(good_trainloader):

            #calculate output
            output = paracrypt_good(x_train)

            #calculate loss
            loss = loss_fn(output,y_train.reshape(-1,1))

            #Calculation of accuracy and other things
            predicted = paracrypt_good(good_trainset.x)
            acc = (predicted.reshape(-1).detach().numpy().round() == b_good).mean()


            #backprop
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if i%50 == 0:
            losses.append(loss)
            accur.append(acc)
            
            paracrypt_good.eval()
            
            trick = torch.from_numpy(np.eye(dim)).to(torch.float32)
            s_candidate = paracrypt_good(trick).reshape(-1).detach().numpy().round()
            print("s' : {b}".format(b = s_candidate.astype(np.int16)))
            
            paracrypt_good.train()
            
            print("epoch {}\tloss : {}\t accuracy : {}".format(i,loss,acc))
            data_file.write("epoch {}\tloss : {}\t accuracy : {} \n".format(i,loss,acc))



s  : [1 0 1 1 0 0 1 1 0 1 1 0 1 0 0 0]
s' : [1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1]
epoch 0	loss : 0.6817697882652283	 accuracy : 0.512206251425964


KeyboardInterrupt: 

In [143]:
torch.save(paracrypt_good.state_dict(), "./experiment-{a}/good_paracrypt.pth".format(a = counter))

### Step 7: Starting the Training

In [71]:
paracrypt = parametric_cryptnet(parametric)
learning_rate = 0.01
epochs = 900
optimizer = torch.optim.SGD(paracrypt.parameters(),lr=learning_rate)
loss_fn = nn.BCELoss()

accur = []
losses = []

with open("./experiment-{a}/training.txt".format(a = counter), "w") as data_file:
    for i in range(epochs):
        for j,(x_train,y_train) in enumerate(trainloader):

            #calculate output
            output = paracrypt(x_train)

            #calculate loss
            loss = loss_fn(output,y_train.reshape(-1,1))

            #Calculation of accuracy and other things
            predicted = paracrypt(trainset.x)
            acc = (predicted.reshape(-1).detach().numpy().round() == b).mean()


            #backprop
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if i%50 == 0:
            losses.append(loss)
            accur.append(acc)
            pred_good = paracrypt(good_trainset.x)
            acc_good = (pred_good.reshape(-1).detach().numpy().round() == b_good).mean()
            pred_bad = paracrypt(bad_trainset.x)
            acc_bad = (pred_bad.reshape(-1).detach().numpy().round() == b_bad).mean()
            print("epoch {}\tloss : {}\t accuracy : {} good_acc : {} , bad_acc : {}\n".format(i,loss,acc, acc_good, acc_bad))
            data_file.write("epoch {}\tloss : {}\t accuracy : {} good_acc : {} , bad_acc : {}\n".format(i,loss,acc, acc_good, acc_bad))
            
        

epoch 0	loss : 0.6966257095336914	 accuracy : 0.5056 good_acc : 0.5095824777549623 , bad_acc : 0.49432739059967584

epoch 50	loss : 0.6932664513587952	 accuracy : 0.5244 good_acc : 0.526694045174538 , bad_acc : 0.5210696920583469

epoch 100	loss : 0.6892038583755493	 accuracy : 0.5342 good_acc : 0.539812913529546 , bad_acc : 0.5332252836304701

epoch 150	loss : 0.6815876960754395	 accuracy : 0.5455 good_acc : 0.5467716176135067 , bad_acc : 0.5364667747163695

epoch 200	loss : 0.6714800596237183	 accuracy : 0.5499 good_acc : 0.5529317818845539 , bad_acc : 0.5453808752025932

epoch 250	loss : 0.6598396301269531	 accuracy : 0.5582 good_acc : 0.5597764088523842 , bad_acc : 0.5453808752025932

epoch 300	loss : 0.647910475730896	 accuracy : 0.5653 good_acc : 0.5660506502395619 , bad_acc : 0.5470016207455429

epoch 350	loss : 0.6370813846588135	 accuracy : 0.5715 good_acc : 0.5726671229751312 , bad_acc : 0.5567260940032415

epoch 400	loss : 0.6257693767547607	 accuracy : 0.5774 good_acc : 0.5

### Step 7.5: Save the model

In [47]:
torch.save(paracrypt.state_dict(), "./experiment-{a}/paracrypt.pth".format(a = counter))

### Step 8: Train from checkpoint

In [115]:
paracrypt = parametric_cryptnet(parametric)
paracrypt.load_state_dict(torch.load("./experiment-{a}/paracrypt.pth".format(a = counter)))
paracrypt.train()

learning_rate = 0.01
epochs = 400
optimizer = torch.optim.SGD(paracrypt.parameters(),lr=learning_rate)
loss_fn = nn.BCELoss()

accur = []
losses = []

with open("./experiment-{a}/training.txt".format(a = counter), "w") as data_file:
    for i in range(epochs):
        for j,(x_train,y_train) in enumerate(trainloader):

            #calculate output
            output = paracrypt(x_train)

            #calculate loss
            loss = loss_fn(output,y_train.reshape(-1,1))

            #Calculation of accuracy and other things
            predicted = paracrypt(trainset.x)
            acc = (predicted.reshape(-1).detach().numpy().round() == b).mean()

            #backprop
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if i%50 == 0:
            losses.append(loss)
            accur.append(acc)
            print("epoch {}\tloss : {}\t accuracy : {}".format(i,loss,acc))
            data_file.write("epoch {}\tloss : {}\t accuracy : {}\n".format(i,loss,acc))

epoch 0	loss : 0.216057687997818	 accuracy : 0.8766


KeyboardInterrupt: 

### Step 9: Evaluating the Accuracy of the Nueral Network

In [48]:
experiment_num = 32 #PLEASE CHANGE THIS VALUE|

In [49]:
#We need to load from A, b, and blah blah based on the experiment number
trick = torch.from_numpy(np.eye(dim)).to(torch.float32)

import json

data_dict = {}

with open("./experiment-{a}/secret.json".format(a = experiment_num), "r") as data_file:
    data_dict = json.load(data_file)
    
A = np.array(data_dict["A"])
b = np.array(data_dict["b"])
s = np.array(data_dict["s"])
p = data_dict["p"]

parametric = {}

with open("./experiment-{a}/parametric.json".format(a = experiment_num), "r") as data_file:
    parametric = json.load(data_file)
    
k = parametric["s"]

paracrypt = parametric_cryptnet(parametric)
paracrypt.load_state_dict(torch.load("./experiment-{a}/paracrypt.pth".format(a = experiment_num))) #CHANGE THIS
paracrypt.eval()

s_candidate = paracrypt(trick).reshape(-1).detach().numpy().round()

print("s' : {b}".format(b = s_candidate.astype(np.int16)))
print("s  : {a}".format(a = s))

s' : [0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 0]
s  : [0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 0]


In [50]:
#We need to load from A, b, and blah blah based on the experiment number
trick = torch.from_numpy(np.eye(dim)).to(torch.float32)

import json

data_dict = {}

with open("./experiment-{a}/secret.json".format(a = experiment_num), "r") as data_file:
    data_dict = json.load(data_file)
    
A = np.array(data_dict["A"])
b = np.array(data_dict["b"])
s = np.array(data_dict["s"])
p = data_dict["p"]

parametric = {}

with open("./experiment-{a}/parametric.json".format(a = experiment_num), "r") as data_file:
    parametric = json.load(data_file)
    
k = parametric["s"]

paracrypt_good = parametric_cryptnet(parametric)
paracrypt.load_state_dict(torch.load("./experiment-{a}/good_paracrypt.pth".format(a = experiment_num))) #CHANGE THIS
paracrypt.eval()

s_candidate = paracrypt(trick).reshape(-1).detach().numpy().round()

print("s' : {b}".format(b = s_candidate.astype(np.int16)))
print("s  : {a}".format(a = s))

FileNotFoundError: [Errno 2] No such file or directory: './experiment-32/good_paracrypt.pth'

In [118]:
#Calculate the Hamming Weight
import math
hamming_weight = np.mod(A @ s_candidate + b, 2).sum()
print("hamming weight : {a}".format(a = hamming_weight))

#Need to figure out the formula to calculate the bounds tomorrow, and make this even more
#parametric to work faster and figure out more

m = 4 * k * math.pow(0.5 - p, -2)
bound = m * p + math.pow(k * m, 0.5)

with open("./experiment-{a}/results.txt".format(a = experiment_num), "w") as data_file:
    if hamming_weight < sample_size * p + bound:
        print("Prediction was correct!")
        data_file.write("Prediction was correct!\n")
    else:
        print('Wrong candidate. Try again!')
        data_file.write('Wrong candidate. Try again!\n')
        
    data_file.write("s' : {b}\n".format(b = s_candidate))
    data_file.write("s : {a}\n".format(a = s))
    data_file.write("hamming weight : {a}\n".format(a = hamming_weight))
    
    
    
    

    

hamming weight : 1234.0
Prediction was correct!


### Step 10: Delete all the models(God forbid)

In [95]:
!rm -rf ./experiment-*

In [84]:
##Test bench code
import numpy as np
e = np.random.binomial(1, 0.125, 1000)
A = np.random.randint(0, 2, size=(1000, 32))

print(A)
goods = np.where(e == 0)[0]
bads = np.where(e == 1)[0]
print(goods)
print(bads)

A_good = A[goods, :]
print(A_good.shape)

A_bad = A[bads, :]
print(A_bad.shape)

b_good = b[goods]
print(b_good)

##So this code actually works


[[0 0 1 ... 1 1 1]
 [1 1 0 ... 1 1 1]
 [1 1 0 ... 0 1 0]
 ...
 [1 0 1 ... 0 1 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 1 0]]
[  0   1   2   4   5   6   7   8   9  11  12  13  14  15  17  19  20  21
  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  39  40
  41  42  44  45  46  47  48  49  50  51  53  54  57  58  59  60  61  62
  63  65  66  67  69  70  71  73  74  75  76  77  78  80  81  82  83  84
  85  87  88  89  90  91  92  93  94  95  96  97  98  99 100 102 103 104
 105 106 108 109 110 111 112 113 115 116 117 118 119 120 121 122 123 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 168 169 170 171 172 174 175 176 178 179 180 182 183
 184 185 186 187 188 190 192 193 194 195 196 197 198 199 200 201 202 203
 204 205 206 207 208 209 210 211 212 213 214 216 217 218 219 220 221 222
 223 224 225 226 227 228 230 231 232 233 234 235 236 238 239 240 241 242
 243